In [1]:
import gurobipy as gp
from gurobipy import GRB

# Initialize the model
model = gp.Model("supply_chain")

# Define sets
manufacturers = ['TechHub', 'Gadgetville']
distributors = ['E-Tech Depot', 'Gizmo Central', 'Electro Hub', 'Digital Warehouse']
retailers = ['Tech Solutions', 'Gadget World', 'Electronics Emporium', 'Digital Dreams', 'Gizmo Galaxy', 'Innovation Station']

# Define capacities and demands
supply  = {'TechHub': 7500, 'Gadgetville': 9500}
throughput_capacity = {'E-Tech Depot': 4000, 'Gizmo Central': 6000, 'Electro Hub': 5500, 'Digital Warehouse': 4500}
demand = {'Tech Solutions': 2500, 'Gadget World': 1800, 'Electronics Emporium': 3000, 
                    'Digital Dreams': 2700, 'Gizmo Galaxy': 4000, 'Innovation Station': 2200}

# Define transportation costs
cost_manufacturer_distribution  = {
    ('TechHub', 'E-Tech Depot'): 2.0, 
    ('TechHub', 'Gizmo Central'): 1.5, 
    ('TechHub', 'Electro Hub'): 2.5, 
    ('TechHub', 'Digital Warehouse'): 2.0,
    ('Gadgetville', 'Gizmo Central'): 1.0, 
    ('Gadgetville', 'Electro Hub'): 1.5, 
    ('Gadgetville', 'Digital Warehouse'): 2.0
}

cost_distribution_retailer = {
    ('E-Tech Depot', 'Tech Solutions'): 2.5, 
    ('E-Tech Depot', 'Electronics Emporium'): 3.0, 
    ('E-Tech Depot', 'Digital Dreams'): 4.0, 
    ('E-Tech Depot', 'Innovation Station'): 2.0,
    ('Gizmo Central', 'Tech Solutions'): 1.5, 
    ('Gizmo Central', 'Gadget World'): 2.0, 
    ('Gizmo Central', 'Electronics Emporium'): 1.0, 
    ('Gizmo Central', 'Digital Dreams'): 2.0,
    ('Electro Hub', 'Tech Solutions'): 1.0, 
    ('Electro Hub', 'Gadget World'): 1.5, 
    ('Electro Hub', 'Electronics Emporium'): 1.0, 
    ('Electro Hub', 'Gizmo Galaxy'): 1.0,
    ('Electro Hub', 'Innovation Station'): 3.0,
    ('Digital Warehouse', 'Electronics Emporium'): 3.5, 
    ('Digital Warehouse', 'Digital Dreams'): 2.0, 
    ('Digital Warehouse', 'Gizmo Galaxy'): 1.0,
    ('Digital Warehouse', 'Innovation Station'): 3.0
}

cost_manufacturer_retailer = {
    ('TechHub', 'Tech Solutions'): 2.5, 
    ('TechHub', 'Gadget World'): 3.0, 
    ('TechHub', 'Electronics Emporium'): 3.5, 
    ('TechHub', 'Digital Dreams'): 4.0, 
    ('TechHub', 'Gizmo Galaxy'): 2.0,
    ('TechHub', 'Innovation Station'): 2.5,
    ('Gadgetville', 'Tech Solutions'): 3.0, 
    ('Gadgetville', 'Gadget World'): 2.0, 
    ('Gadgetville', 'Electronics Emporium'): 2.5, 
    ('Gadgetville', 'Digital Dreams'): 2.5, 
    ('Gadgetville', 'Gizmo Galaxy'): 1.5,
    ('Gadgetville', 'Innovation Station'): 2.0
}

Restricted license - for non-production use only - expires 2025-11-24


In [2]:
model = gp.Model("TransportationCostMinimization")

# Decision variables
x_mw = {}
x_wr = {}
x_mr = {}

# Define decision variables
for m in manufacturers:
    for w in distributors:
        x_mw[m, w] = model.addVar(vtype=GRB.CONTINUOUS, name=f'x_{m}_{w}')

for w in distributors:
    for r in retailers:
        x_wr[w, r] = model.addVar(vtype=GRB.CONTINUOUS, name=f'x_{w}_{r}')

for m in manufacturers:
    for r in retailers:
        x_mr[m, r] = model.addVar(vtype=GRB.CONTINUOUS, name=f'x_{m}_{r}')


In [3]:
# Objective function: minimize total transportation cost
model.setObjective(
    gp.quicksum(cost_manufacturer_distribution[m, w] * x_mw[m, w] for m in manufacturers for w in distributors if (m, w) in cost_manufacturer_distribution) +
    gp.quicksum(cost_distribution_retailer[w, r] * x_wr[w, r] for w in distributors for r in retailers if (w, r) in cost_distribution_retailer) +
    gp.quicksum(cost_manufacturer_retailer[m, r] * x_mr[m, r] for m in manufacturers for r in retailers if (m, r) in cost_manufacturer_retailer),
    GRB.MINIMIZE
)


In [4]:
# Supply constraints
for m in manufacturers:
    model.addConstr(
        gp.quicksum(x_mw[m, w] for w in distributors if (m, w) in cost_manufacturer_distribution) +
        gp.quicksum(x_mr[m, r] for r in retailers if (m, r) in cost_manufacturer_retailer) <= supply[m],
        f'Supply_{m}'
    )

# Demand constraints
for r in retailers:
    model.addConstr(
        gp.quicksum(x_wr[w, r] for w in distributors if (w, r) in cost_distribution_retailer) +
        gp.quicksum(x_mr[m, r] for m in manufacturers if (m, r) in cost_manufacturer_retailer) == demand[r],
        f'Demand_{r}'
    )

# Flow conservation constraints
for w in distributors:
    model.addConstr(
        gp.quicksum(x_mw[m, w] for m in manufacturers if (m, w) in cost_manufacturer_distribution) ==
        gp.quicksum(x_wr[w, r] for r in retailers if (w, r) in cost_distribution_retailer),
        f'Flow_{w}'
    )

# Throughput capacity constraints
for w in distributors:
    model.addConstr(
        gp.quicksum(x_mw[m, w] for m in manufacturers if (m, w) in cost_manufacturer_distribution) <= throughput_capacity[w],
        f'Throughput_{w}'
    )


In [5]:
# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution:")
    for v in model.getVars():
        if v.x != 0:
            print(f'{v.varName}: {v.x}')
    print(f'Total Transportation Cost: {model.objVal}')
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 44 columns and 79 nonzeros
Model fingerprint: 0x27a45c33
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+04]
Presolve removed 1 rows and 8 columns
Presolve time: 0.01s
Presolved: 15 rows, 36 columns, 78 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2000000e+04   2.025000e+03   0.000000e+00      0s
      14    3.5100000e+04   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.510000000e+04
Optimal Solution:
x_Gadgetville_Gizmo Central: 3000.0
x_Gizmo Central_Electronics Emporium: 3000.0
x_TechHub_Tech Sol